Documentation :

- V.Small Data about 21 Failed Companies

- Extracted city from the **Based in** column and removed it

- Extracted *year* from **Launch** and **Closure** 

- Replaced null values in **Number of investors** with min of all values

- Removed **Acquisition date,Accelator Program,Founding rounds amount,Last Funding,Last Funding Date,Last Funding type** as they have more than 50% missing values

- Modified **Employee count** to **Avg Employee Count**

- Modified **Acquisition Status** to Yes/No (binary features)

- Replaced **-** with *no_data*

In [175]:
import pandas as pd
import numpy as np
import re 

In [176]:
file_path = '/content/failure_startup_autopsy_data.csv'
df = pd.read_csv(file_path)

original_df = df.copy()

In [177]:
df.head()

,Unnamed: 0,Company_Name,Based in,Company_type,Business_type,Description,Launch,Closure,Duration (months),Reason,Failure categories,Number of investors,Accelerator program,Acquisition status,Acquisition date,Employee count,Founders,Total funding,Number of funding rounds,Funding round(s) amount(s),Last funding,Last funding date,Last funding type,Funding Status
0,0,Amiloom,"San Francisco, California, United States",Information Technology,B2C,Device forms network of people w/common friends,1 December 2012,1 October 2014,"1 year, 10 months",Cannot be clearly explained in 3 seconds,Poor Marketing,-,-,-,-,1-10,Anders Hsi,"$1,713 pledged of $108,127 goal",1,-,"$1,713 pledged of $108,127 goal",20 November 2014,Crowfunding,Crowfunding
1,1,Backfence,"Mclean, Virginia, United States","Communities, Internet, Web Hosting",B2C,create hyper-local community sites,1 January 2004,1 July 2007,"3 years, 6 months",internal problems made it impossible to reach ...,​,2,-,-,-,11-50,Mark Potts,"$3,000,000",1,​,"$3,000,000",13 October 2006,Venture - Series Unknown,Venture - Series Unknown
2,2,Circa_News,"San Francisco, California, United States","Mobile, News, Apps",B2C,Mobile news app,1 December 2011,1 June 2015,"3 years, 6 months","Lack of funding, no buyers",No Financing or Investor Interest,39,-,Acquired by Sinclair Broadcast Group for undis...,9 December 2015,11-50,"Arsenio Santos, Ben Huh, Matt Galligan","$4,737,750",7,"Seed - $1,225,000 , Funding Round - $1,092,750...","$4,737,750",1 March 2014,Seed,M&A
3,3,DeviceFidelity,"Richardson, Texas, United States","Mobile, Wireless, Payments",B2B,Contactless payments / NFC for iphone,1 January 2007,1 February 2015,"8 years, 1 month",Apple built NFC into new product line,​,2,-,Acquired by Kili for undisclosed amount,12 November 2014,11-50,"Amitaabh Malhotra, Deepak Jain","$9,646,024",4,-,"$9,646,024",4 March 2013,Debt Financing,M&A
4,4,Gumroad_,"San Francisco, California, United States","Content, Creative Agency, E-Commerce",B2C,Owns and operates an online service through wh...,1 January 2011,1 January 2019,8 years,It took years for me to realize my pursuit of ...,​,10,-,-,-,11-50,Sahil Lavingia,"$8,100,000",2,"Seed - $1,100,000, Series A - $7,000,000","$8,100,000",8 February 2012,Series A,Series A


In [178]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  21 non-null     int64 
 1   Company_Name                21 non-null     object
 2   Based in                    21 non-null     object
 3   Company_type                21 non-null     object
 4   Business_type               21 non-null     object
 5   Description                 21 non-null     object
 6   Launch                      21 non-null     object
 7   Closure                     21 non-null     object
 8   Duration (months)           21 non-null     object
 9   Reason                      21 non-null     object
 10  Failure categories          21 non-null     object
 11  Number of investors         21 non-null     object
 12  Accelerator program         21 non-null     object
 13  Acquisition status          21 non-null     object
 

In [179]:
def extract_city(text):

  city = text.replace("United States"," ").strip().split(',')[0]
  return city

In [180]:
df['City'] = df['Based in'].apply(extract_city)

In [181]:
def extract_year(text):

  year = re.search(r"\d{4}",text).group(0)
  return year


df['Launch_Year'] = df['Launch'].apply(extract_year) 
df['Close Year']  = df['Closure'].apply(extract_year)

In [182]:
df['Accelerator program'].value_counts()

-                              17
Y Combinator & 500 Startups     1
500 Startups                    1
Y Combinator                    1
Techstars 2016 cohort           1
Name: Accelerator program, dtype: int64

In [183]:
df['Acquisition_status(y/n)'] = df['Acquisition status'].apply(lambda x: "no" if x=='-' else "yes")

In [184]:
remove_cols = ['Unnamed: 0','Based in','Launch','Closure','Duration (months)','Accelerator program','Acquisition status','Acquisition date','Funding round(s) amount(s)', 'Last funding date', 'Last funding type']
df.drop(columns = remove_cols,axis=1,inplace=True)

In [185]:
df.drop(columns = [df.columns[-6]],axis = 1,inplace=True)

- modifying employee count

In [186]:
def avg_employee_count(count):

  min_ = float(count.split('-')[0])
  max_ = float(count.split('-')[1])

  return (min_ + max_)/2.0
  


In [187]:
df['avg_employee_count'] = df['Employee count'].apply(avg_employee_count)

In [188]:
df['avg_employee_count']

0        5.5
1       30.5
2       30.5
3       30.5
4       30.5
5       75.5
6      175.5
7       30.5
8        5.5
9     7500.5
10     175.5
11      75.5
12      75.5
13       5.5
14      30.5
15      30.5
16       5.5
17      75.5
18    3000.5
19      30.5
20       5.5
Name: avg_employee_count, dtype: float64

In [189]:
df.replace('-','no_data',inplace=True)

In [190]:
space = df['Failure categories'].value_counts().index[0]

df.replace(space,"no_data",inplace=True)

In [191]:
min_investors = df['Number of investors'].min()
df['Number of investors'].replace("no_data",min_investors,inplace=True)

In [193]:
df.drop(columns = ['Employee count'],axis = 1,inplace=True)

In [213]:
def extract_funding(fund):
   
  if fund != "no_data":

    funding = float(fund.replace(',','').replace('$',''))

    return funding

In [203]:
df['Total funding'][1:]

1       $3,000,000
2       $4,737,750
3       $9,646,024
4       $8,100,000
5      $65,890,945
6     $118,500,000
7       $8,100,000
8       $3,200,000
9      $21,600,000
10     $88,600,000
11    $118,100,000
12     $66,150,000
13         $15,000
14     $62,100,000
15     $14,950,000
16         no_data
17      $1,800,000
18    $194,600,000
19     $73,500,000
20         no_data
Name: Total funding, dtype: object

In [214]:
df['total_funding(in $)'] = df['Total funding'][1:].apply(extract_funding)

In [215]:
df['total_funding(in $)']

0             NaN
1       3000000.0
2       4737750.0
3       9646024.0
4       8100000.0
5      65890945.0
6     118500000.0
7       8100000.0
8       3200000.0
9      21600000.0
10     88600000.0
11    118100000.0
12     66150000.0
13        15000.0
14     62100000.0
15     14950000.0
16            NaN
17      1800000.0
18    194600000.0
19     73500000.0
20            NaN
Name: total_funding(in $), dtype: float64

In [217]:
min_fund = df['total_funding(in $)'].min()
df['total_funding(in $)'].fillna(min_fund,inplace=True)

In [221]:
df.drop(columns = ['Total funding'],axis = 1,inplace=True)

In [223]:
df.to_csv("cleaned_failure_startup_autopsy_data.csv")

- failed regex experiments

In [ ]:
#import re
#s1 = re.sub(r'United States\s+',"",s)

In [ ]:
#s2 = df['Launch'].str.extract(r'^(\d{4})',expand=False)

##extr = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)

In [ ]:
#s2 = df['Launch'][0]

In [ ]:
#year = re.search(r"\d{4}", df['Launch'][0]).group(0)

In [ ]:
#year

In [ ]:
#s2_ = re.sub(r'\d+ .* \d{4}',"",(s2))

In [ ]:
#s2_ = df['Launch'][0].replace(r"\d+.*")

''

In [47]:
#s2 = df['Duration (months)'][0]

In [ ]:
#s2

In [90]:
#year = int(re.search(r"(\d?) (years?)",s2).group(1))
#month = re.search(r"(\d+) (months?)",s2)

In [ ]:
#year

In [ ]:
#month

In [92]:
#month = re.search(r"((\d+) (months?))?",s2).groups()

In [ ]:
#month